# Введення в реляційні бази даних.
# PostgreSQL + Python

---

## лекція 3: Об'єднання таблиць (JOIN) і підзапити

In [ ]:
from python_postgresql import execute_query, execute_read_query, connection
print("лекція 3: Об'єднання таблиць (JOIN) і підзапити")

### JOIN - поєднання таблиць

До цього ми з Вами працювали лише з однією таблицею. Але - це на завжди зручно і можливість мати поєднання декількох таблиць для отримання фінального результату - це потужна річ, яка забезпечується оператором JOIN. Докладно про особливості використання JOIN в PostgreSQL - [тут](https://www.postgresql.org/docs/14/queries-table-expressions.html#QUERIES-FROM).

Якщо просто - ця операція дає можливість поєднати декілька таблиць у одну велику таблицю за якимось правилом. Перд тім як ми перейдемо до коду - які бувають поєднання (ми зараз не про всі можливі, а про ті які вживаються частіше).

Правило поєднання - це відповідність атрибута однієї таблиці атрибуту іншої таблиці. Наприклад - у нас є users.id (це первинний ключ для таблиці users) і в таблиці emails є атрибут user_id - (вторинний ключ, який посилається на id таблиці users). Ми можемо поєднати users і emails за принципом users.id = emails.user_id. Трохи докладніше розберемо на прикладі далі.

![як працює JOIN](media/sql-table-joins.png)

У кожній операції поєднання таблиць беруть участь дві таблиці - "ліва" і "права" (звісно ми можемо виконати декілька операцій об'єднання таблиць підряд і поєднати більше ніж дві таблиці, але навчатись краще на прикладі з лише двома таблицями). Операція приєднання - не симетрична, тобто результат може залежати від "зміни місць".

---

#### INNER JOIN

- в результаті будуть тільки ті рядки, для яких той самий принцип поєднання, за яким ми приєдуємо таблиці, присутній для двох таблиць. Давайте уявимо що нам потрібна таблиця яка включає всі замовлення (orders) клієнта Wartian Herkku (customer_id = 'WARTH')  з даними про компанію яка займалась доставкою (таблиця shippers) цього замовлення до клієнта. Тобто, ми хочемо зробити наступне поднання талиць:
-  ми приєднуємо до orders ("ліва таблиця") таблицю shippers ("права таблиця") за принципом orders.ship_via = shippers.shipper_id, обираємо лише рядки для яких orders.customer_id = 'WARTH'
Як при цьому працює поєднання?
1) Береться рядок з "лівої" таблиці і СУБД дивиться - чи це потрібний для нас рядок (orders.order_id = 'WARTH')? Якщо умова не виконується - відкидає його і переходи до обробки наступного рядка. Якщо умова виконується -  СУБД дивиться чи є атрибут orders.ship_via (в різних випадках він може бути і не бути - можливо у нас бувають умови самовивозу замовлень для клієнтів і тоді компанії-доставника просто не існує для цього замовлення - тобто значення NULL - "нічого").
Якщо атрибута такого немає, то цей рядок відкидається і не попадає у фінальний набір. Переходимо до іншого рядка "лівої" таблиці.
2) Якщо у рядка "лівої" таблиці є атрибут для поєднання, то починаємо дивитись рядки "правої" таблиці. Дивимось саме той атрибут, по якому ця таблиця повинна бути приєднана до "лівої". В нашому випадку - це атрибут shippers.shipper_id. Як тільки знаходиться рядок, для якого умова поєднання True (тобто orders.ship_via = shippers.shiper_id - поєднаний рядок (атрибути з таблиці orders з рядка першого кроку) + атрибути знайденого рядка з таблиці shippers на цьому кроці включається в фінальний набір.
3) В нашому випадку значенню orders.ship_via відповідає лише одне значення з таблиці shippers.shiper_id (зрозуміло що у одного замовлення може бути один перевізник). І це зафіксовано в обмеженнях на дані для наших таблиць - ми маємо відношення "один до багатьох" між таблицями shippers та orders (у кожного замовлення може бути лише один перевізник, але один перевізник може бути у багатьох замовлень). Тому СУБД, знаючи про це обмеження і знайшовши відповідність для поєднання рядків - зупиняє пошук і переходе до обробки інших рядків "лівої" таблиці. Але, якщо б ми використовували іншу модель обмежень даних (наприклад - замовлення такі великі і можуть доставлятись різними перевізниками навіть у межах одного замовлення - при цьому "права" таблиця малаб іншу структуру - ми будемо про це говорити докладно пізніше), тоді СУБД переходять до інших рядків "правої" таблиці. Якщо знову находять рядок в цій таблиці, для якого атрибут приєднання дорівнює знайденому на кроці 1 атрибуту приеднання "правої" таблиці, то і ці рядки включаються до фінального набору - тобто в цьому разі наш поєднаний рядок ПОВТОРИТЬ всі атрибути з таблиці orders (той рядок, який ми обрали на етапі 1 і він став першою частиною результата на етапі 2) і додасть до нього атрибути з рядка таблиці shippers, який ми знайшли на цьому кроці.
4) Що буде - якщо для атрибута orders.ship_via рядка, який ми обрали на першому етапі ми не знайдемо ні однієї відповідності атрибуту shippers.shipper_id з таблиці shipper (в нашому випадку це неможливо, але в загальному - це реально)? Рядок з таблиці НЕ БУДЕ ВКЛЮЧЕНО до фінального набору.

Тобто - при використанні INNER JOIN у фінальну вибірку включаються лише ті рядки з обох таблиць, для яких ОБОВ'ЯЗКОВО є виконання умови поєднання.

In [ ]:
operation_query = """
SELECT *
FROM orders
INNER JOIN shippers ON orders.ship_via = shippers.shipper_id
WHERE orders.customer_id = 'WARTH';
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Важливо запам'ятати, що в результати INNER JOIN не попадуть рядки, для яких не виконується умова поєднання ДЛЯ ОБОХ ТАБЛИЦЬ.
Ви можете використовувати JOIN замість INNER JOIN - за замовченням розуміється саме INNER JOIN.

Двайте зробимо ще один запит для закріплення матеріалу.
Давайте виберео всі замовлення того ж клієнта Wartian Herkku (customer_id = 'WARTH') за весь час і отримаємо всі його замовлення і їх сумми.

In [ ]:
operation_query = """
SELECT orders.order_id, SUM(order_details.unit_price * order_details.quantity * ( 1 - order_details.discount))
FROM orders
INNER JOIN order_details ON orders.order_id = order_details.order_id
WHERE orders.customer_id = 'WARTH'
group by orders.order_id;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

А тепер зробимо поєднання більше ніх двох таблиць.
Нам необхідна талиця, яка надасть нам інформацію про дату замовлення, номер замовлення, назву товара в замовленні, його вартість, дисконт, кількість у замовленні і сумму по товару, а також країну в яку ми все це відправляли. Тут точно буде поєнання більше ніж двох таблиць.

In [ ]:
operation_query = """
SELECT orders.order_date, orders.order_id, products.product_name, order_details.unit_price, order_details.discount, order_details.quantity, ( order_details.unit_price * (1 - order_details.discount) * order_details.quantity) AS total_price, orders.ship_country
FROM orders
INNER JOIN order_details ON orders.order_id = order_details.order_id
INNER JOIN products ON products.product_id = order_details.product_id;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

---

#### OUTER JOIN (LEFT JOIN, RIGHT JOIN)

Давайте уявимо що ми хочемо вивести таблицю яка буде включати ВСІ рядки для "лівої" таблиці і, якщо для якогось з ціх рядків не виконується умова поєднання у "правій" таблиці - просто мати в доповнених ячейках NULL. Для нас не підійде поєднання INNER JOIN. Тому що там можуть бути не всі користувачі. Передивіться крок 4 попереднього пояснення - якщо для якогось користувача не буде ні однієї відповідності у "правій" таблиці - то цей рядок не буде включений у фінальний результат.

В цьому випадку необхідно використовувати OUTER JOIN - в одному із його варіантів - LEFT JOIN або RIGHT JOIN (трошки пізніше - про різницю між ними).

Алгоритм включення рядків до фінального результату - інший. Коротко:
- для LEFT JOIN
    - беремо рядок "лівої таблиці". Для атрибута приеднання в "правій таблиці" починаємо шукати співпадіння з її атрибутом приеднання. Якщо співпадіння є - включаемо поєднаний рядок у фінальний результат. Якщо співпадіння немає - ми все одно включаємо в фінальний результат поєднаний рядок, але всі атрибути з "правої таблиці" які повинні війти у фінальний результат, заповнюемо значенням NULL ("нічого немає").
- для RIGHT JOIN
    - все теж саме, що і для LEFT JOIN, але в якості "лівої таблиці" - тобто тієї, з якої всі рядки попадуть у фінальний результат, береться та, що стоїть справа))). Тобто - те ж саме, просто погляд з іншого боку. Якщо просто поміняти таблиці у виразі містами і використати LEFT JOIN - буде те ж саме що не міняти і виконати RIGHT JOIN.


LEFT JOIN (RIGHT JOIN) - не завжди буде відрізнятись від INNER JOIN. Якщо для ВСІХ рядків "правої" таблиці будуть існувати відповідні рядки "лівої" таблиці - то обидва ціх поєднання дадуть однаковий результат.

Давайте спробуємо знайти компанії (customers) на яких взагалі немає замовлень (orders).
Для цього нам треба поєднати ці дві таблиці саме через LEFT JOIN (тому що тоді можуть з'явитись рядки де всі артибути стосовно замовлень будуть NULL), і відфільтрувати такі рядки

In [ ]:
operation_query = """
SELECT customers.customer_id, customers.company_name, orders.order_id
FROM customers
LEFT JOIN orders ON orders.customer_id = customers.customer_id
WHERE orders.order_id IS NULL;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Самостійно замініть LEFT JOIN на INNER JOIN і порівняйте результати запитів.
Пропоную Вам знайти робітників компанії, які не згенерували ні одного замовлення).

#### FULL JOIN

Після вивчення LEFT JOIN і RIGHT JOIN тут все просто. У фінальний результат включаються ВСІ рядки з обох таблиць - і з "лівої таблиці" і з "правої". Якщо в якихось випадках вираз поєднання (left_table.attribute_X == right_table.attribute_Y) має значення False (тобто для якогось рядка немає відповідного по цій парі атрибутів рядка з іншої таблиці) - то рядки все одно вносяться у фінальний результат, але там де немає відповідності - атрибути заповнюються значенням NULL. Тобто - це симетрична операція, і в загальному випадку у фінальному результаті будуть атрибути із значенням NULL як у правій частині фінального результату, так і в лівій.

#### CROSS JOIN

CROSS JOIN використовується нечасто. Якщо говорити про це строгою мовою визначень - це декартове множення таблиць. Тобто - сполучену таблицю утворюють всі можливі поєднання рядків з T1 і T2, а набір її стовпців поєднує у собі стовпці  T1 з наступними стовпцями T2. Якщо таблиці містять N і M рядків, з'єднана таблиця міститиме N * M рядків.

```FROM T1 CROSS JOIN T2``` рівнозначно ```FROM T1 INNER JOIN T2 ON TRUE```. Цей запис також рівнозначний ```FROM T1, T2```

#### USING та NATURAL JOIN

Про "синтаксичний цукор".
Нагадую, що цей термін використовують коли хочуть описати якусь спеціальну конструкцію мови, яка не додає функціональності - тобто все це є можна зробити і за допомогою інших конструкцій - але робить наш код коротшим або більш виразним.
Зараз - саме про такі конструкції.

USING - зручно використовувати коли Ви проводите поєднання різних тались по атрибутам які мають однакові назви. При цьому не потрібно вказувати назву таблиці - тільки назву стовпця - і це робить вираз більш лаконічним.
Приклад:

In [ ]:
operation_query = """
SELECT order_id, product_name, category_name
FROM orders
JOIN order_details USING(order_id)
JOIN products USING(product_id)
JOIN categories USING(category_id)
LIMIT 10;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Пропоную Вам для порівняння та відпрацювання навичок переписати цей же запит звичайним чином - викорстовуючи __ON__.

NATURAL JOIN - ще розвиває цб ідею.
Просто вказуєш таблицю - і СУБД самостійно шукає стовпці з однаковими назвами і з'єднує таблиці за відповідністю ціх стовпців.
Приклад запиту:

In [ ]:
operation_query = """
SELECT order_id, customer_id, first_name, last_name, title
FROM orders
NATURAL JOIN employees
LIMIT 10;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Все дуже коротко і працює, але...
Завжди є але. І, на мій погляд, в цьому випадку дуже істотне.
Стовпці не вказані явним чином. А якщо їх декілька? Ну да це ще не сама проста біда. А якщо, після того як запит написаний і працює, таблиці були змінені і в них були додані стовпці з однаковими назвами? Яка буде поведінка?
Ми з Вами вивчаємо python і, на одній з перших лекцій ми говорили про дзен-python і один з принципів - "Explicit is better than implicit" - "явне краще ніх неявне". Я не рекомендую Вам використовувати NATURAL JOIN саме з цієї причини - він може бути джерелом проблем які важко виявляти.

#### AS - псевдоніми

Ми вже використовували з Вами конструкцію AS.
Вона надає псевдонім якомусь атрибуту (або навіть таблиці чи ще якомусь об'єкту) для зручності використання.
Наприклад запит:
```SQL
SELECT COUNT(*) AS orders_count
FROM orders;
```
сфрмує для нас таблицю з одним рядком і одним стовпчиком в цьому рядку, і цей стовпчик буде мати навзву __orders_count__

AS можна використовувати не скрізь.
Наприклад псевдоніми не можна використовувати в WHERE (HAVING) - тому що  WHERE (HAVING) відпрацьовує до того, як буде виконуватись SELECT - в якому буде відпрацьована команда AS.

Приклад:

In [ ]:
operation_query = """
SELECT category_id, SUM(unit_price * units_in_stock) AS total_price
FROM products
WHERE discontinued <> 1
GROUP BY category_id
HAVING SUM(unit_price * units_in_stock) > 1000
ORDER BY total_price;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Ми не мали можливості використовувати total_price з HAVING, але з GROUP BY - без перешкод. Поексперементуйте с запитом.


Звеніть увагу на псевдоніми - по існують ситуації при яких без них дуже важно (або неможливо) - наприклад при використанні підзапитів до вивчення яких ми перейдемо в наступному розділі.

### Підзапити

[документація тут](https://www.postgresql.org/docs/14/functions-subquery.html)

Інкол запити бувають складними і надто складними. І їх дуже важно одразу написати.

Але, сподіваюсь, ви пам'ятеєте що основа вирішення складних завдань - декомпозиція. Тобто - розбити складне завдання на більш прості складови частини і вирішувати простіші завдання - які потім об'єдуються в базову складну задачу.

Саме так і в написанні складних запитів: їх можна розбивати на більш прості і зрозумілі підзапити і потім конструювати фінальне рішення з ціх підзапитів.

Ми вже говорили про те, що результат виконання будь-якого запиту - таблиця. А раз це таблиця - то ми можемо використовувати до неї весь відомий нам апарат SQL: робити до неї запити, поєднувати з іншими таблицями і т.д. Все це лежить в основі побудови і використання підзапитів.

Необхідно розуміти, що в більшості ситуацій запит з підзапитом може бути переписаний без підзапита (як правило з використанням поєднання таблиць) - але це може бути менш читаємо. Для того щоб зрозуміти - чи треба переписувати запити - необхідно оцінбвати швидкість виконання запиту (вона повинна бути не масимально, а достатньою) і читабельність. Більше того - існує такий елемент СУБД як планувальник, який планує виконання запитів на основі нашого синтаксису і він часто сам перетворює наш запит з підзапитами і запит з поєднанням таблиць. Розумною поведінкою буде використовувати планувальник щоб подивитись на швидкість виконання запитів і потім приймати рішення. Як це робити - ми будемо розмовляти пізніше.

А зараз про синтаксис підзапитів на практиці.
Уявимо, що керівництво нашої компанії планує привітати з якимось святом керівників компаній-постачальників передавши їм подарунки, та не хоче витрачатись на курьерів), а тому хоче використати наших співробітників у тіх містах, де є постачальники і наші співробітники. І хоче отримати від нас перелік всіх компаній-постачальників і контактних осіб, де є наші співробітники. А для цього нам потрібен перелік міст - де є і наші співробітники і наші постачальники.
Спочатку нам необхідно отримати всі міста де є наші співробітники:

```SQL
SELECT DISTINCT city
FROM employees;
```

In [ ]:
operation_query = """
SELECT DISTINCT city
FROM employees;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Тепер треба вибрати компанії-постачальники, які розташовані в ціх містах. Звичайно, це можливо можна зробити так:

In [ ]:
operation_query = """
SELECT company_name, contact_name, contact_title, city
FROM suppliers
WHERE city in ('Redmond', 'London', 'Tacoma', 'Kirkland', 'Seattle');
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Принймі це працює. Але, уявіть, що таких міст не 5, а 500. Не дуже здорово вбивати це руками. Використати підзапит буде значно елегантніше:

In [ ]:
operation_query = """
SELECT company_name, contact_name, contact_title, city
FROM suppliers
WHERE city in (
    SELECT DISTINCT city
    FROM employees
    );
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Все досить просто - підзапит це звичайний запити, який в дужках підставляється в інший запит там, де необхідно використати результат цього підзапиту.

Давайте напишемо звапит який поверне нам всі товари залишки яких на складі менші ниж середні залишки по товару (в грошовому еквіваленті). Такі запити просто писати послідовно. Спочатку - як отримати середні залишки по найменуванню товарів в грошовому еквіваленті:

```SQL
SELECT AVG(units_in_stock * unit_price)
FROM products
```

Ну а потім необхідно вивести всі найменування товарів, залишки по яким менші ніж це значення.
Фінальний запит виглядає так:

In [ ]:
operation_query = """
SELECT product_id, product_name, (units_in_stock * unit_price) AS balances_for_the_amount
FROM products
WHERE (units_in_stock * unit_price) < (
    SELECT AVG(units_in_stock * unit_price)
    FROM products
    )
ORDER BY balances_for_the_amount DESC;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

#### WHERE EXISTS

Аргументом EXISTS є звичайний оператор SELECT, тобто підзапит. Виконавши запит, система перевіряє, чи він повертає рядки в результаті. Якщо він повертає мінімум один рядок, результатом EXISTS буде "true", а якщо не повертає жодного - "false".

Підзапит може звертатися до змінних зовнішнього запиту, які в рамках одного обчислення підзапиту вважаються константами.

Взагалі кажучи, підзапит може виконуватися не повністю, а завершуватися, щойно буде повернуто хоча б один рядок. Тому у підзапитах слід уникати побічних ефектів (наприклад, звернень до генераторів послідовностей); прояв побічного ефекту може бути непередбачуваним.

Так як результат цього виразу залежить тільки від того, повертаються рядки чи ні, але не від їхнього вмісту, список вихідних значень підзапит зазвичай не має значення. Як наслідок, поширена практика, коли перевірки EXISTS записуються у формі EXISTS(SELECT 1 WHERE ...). Однак із цього правила є й винятки, наприклад із підзапитами з INTERSECT.

Підзапит може звертатися до змінних зовнішнього запиту, які в рамках одного обчислення підзапиту вважаються константами.

Давайте розглянемо це на прикладі - знайдемо всіх замовників, замовлення яких коли небудь важили більше ніж 100 кг.
Запит:

In [ ]:
operation_query = """
SELECT company_name, contact_name, contact_title
FROM customers
WHERE EXISTS (
    SELECT 1
    FROM orders
    WHERE customers.customer_id = customer_id AND freight > 100
);
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Давайте розберемо запит.
В тілі підзапиту - ```SELECT 1 ...```  - нам неважливе значення результату, нам важливо є він чи немає. Тому, якщо він є, нас абсолютно влаштовує значення 1.
В тілі підзапиту ми використовуємо custimers.customer_id ```... WHERE customers.customer_id = customer_id ... ```. Це значення береться з тіла "огортаючого запиту" і його треба сприймати як константу.

#### Підзапити з ANY(SOME)\ALL

Пропоную - на прикладі. Давайте оберемо компанії, які замовляли великі замовлення - більше ніж по 50 одиниць товарів одного найменування в одному замовленні.
Одразу - цей запит можна зробити декількома варіантами. Ми зараз вивчаємо підзапити, тому я іду цім шляхом.
Підзапит має вигляд:

```SQL
SELECT customer_id
FROM orders
JOIN order_details USING(order_id)
WHERE quantity > 50;
```

Підзапит поверне нам всі customer_id замовників, які відповідають умові. Таким чином наш повний запит може мати наступний вигляд:

In [ ]:
operation_query = """
SELECT company_name, contact_name, contact_title
FROM customers
WHERE customer_id = ANY(
    SELECT customer_id
    FROM orders
    JOIN order_details USING(order_id)
    WHERE quantity > 50
);
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Тобто - ANY (SOME - синонім) - "будь-який".

Тепер про умову ALL.

Цей квантифікатор вимагає щоб умова порівняння виконувалась не для одного елементу підзапиту (як ANY), а для всіх.